# Wrangle the Raster Data (3 layers)

In [1]:
# Download stored variables from previous notebook

%store -r habitat_suitability_data_dir usfs_grasslands_path 
%store -r comanche_grassland_gdf pawnee_grassland_gdf usfs_grasslands_gdf

In [ ]:
# Prepare for download Part 1 of 1
## Import packages that will help with...

# Reproducible file paths
import os # Reproducible file paths
import pathlib # Find the home folder
from glob import glob  # returns list of paths
import zipfile # Work with zip files

# Find files by pattern
import matplotlib.pyplot as plt # Overlay pandas and xarry plots,Overlay raster and vector data
import rioxarray as rxr # Work with geospatial raster data


# Work with tabular, vector, and raster data
import cartopy.crs as ccrs # CRSs (Coordinate Reference Systems)
import geopandas as gpd # work with vector data
import hvplot.pandas # Interactive tabular and vector data
import hvplot.xarray # Interactive raster
from math import floor, ceil # working with bounds, floor rounds down ciel rounds up
import pandas as pd # Group and aggregate
from rioxarray.merge import merge_arrays # Merge rasters
import xarray as xr # Adjust images
import xrspatial # calculate slope

# Access NASA data
import earthaccess # Access NASA data from the cloud

## 1. POLARIS dataset - download 2 soil variables

In [ ]:
# Practice downloading soil data with random 1x1 tif
# Part 1 of 3
practice_soil_url = (
            "http://hydrology.cee.duke.edu"
            "/POLARIS/PROPERTIES/v1.0"
            "/ph"
            "/mean"
            "/60_100"
            "/lat2829_lon-101-100.tif"
            )

practice_soil_url

In [ ]:
# Practice downloading soil data with random 1x1 tif
# Part 2 of 3

# Connect to raster image
practice_soil_da = rxr.open_rasterio(
    practice_soil_url,
    mask_and_scale=True
).squeeze()

practice_soil_da

In [ ]:
# Practice downloading soil data with random 1x1 tif
# Part 3 of 3

# Plot
practice_soil_da.plot(
    cbar_kwargs={"label": "pH"},
    robust=True
    )
plt.gca().set(
    title='Practice pH on 1x1 tif', 
    xlabel='Longitude',
    ylabel='Latitude',
)
plt.show()

In [7]:
# Set the site parameters
# soil variables
soil_prop = 'ph'
soil_stat = 'mean'
soil_depth = '60-100'
# set up url template
soil_url_template = (
            "http://hydrology.cee.duke.edu"
            "/POLARIS/PROPERTIES/v1.0"
            "/{soil_prop}"
            "/{soil_stat}"
            "/{soil_depth}"
            "/lat{min_lat}{max_lat}_lon{min_lon}{max_lon}.tif"
            )

# bounds_gdfs
chosen_grasslands_bounds_gdfs = [
    comanche_grassland_gdf,
    pawnee_grassland_gdf
]

# output_directory - create data dir for polaris data 
polaris_dir= os.path.join(habitat_suitability_data_dir, 'polaris')
os.makedirs(polaris_dir, exist_ok=True)

In [ ]:
# Iterate through the list of bounding GeoDataFrames (areas of interest)
#for bounds_gdf in chosen_grasslands_bounds_gdfs:

# Get the study bounds
bounds_min_lon, bounds_min_lat, bounds_max_lon, bounds_max_lat = (
comanche_grassland_gdf
.to_crs(4326)
.total_bounds 
)

# List to store cropped DataArrays for the current site
soil_da_list= []

# Loop through bounding box coordinates
for min_lon in range(floor(bounds_min_lon), ceil(bounds_max_lon)):
    for min_lat in range(floor(bounds_min_lat), ceil(bounds_max_lat)):

        # Format the URL with the current coordinates and other parameters
        formated_url = (
        soil_url_template.format( 
            soil_prop = soil_prop, 
            soil_stat = soil_stat, 
            soil_depth = soil_depth,
            min_lat=min_lat , max_lat=min_lat+1,
            min_lon=min_lon, max_lon=min_lon+1 )
        )

        # Connect to the raster image
        soil_da = rxr.open_rasterio(
        formated_url, 
        mask_and_scale=True
        ).squeeze()
        
        # Crop the raster image to the bounds of the study area
        cropped_da = (
        soil_da_list.rio.clip_box(bounds_min_lon, bounds_min_lat, bounds_max_lon, bounds_max_lat)
        )

        # Append the cropped DataArray to the list
        soil_da_list.append(cropped_da)

    # Merge the cropped DataArrays for this site
    merged_da_list = (
    merge_arrays(soil_da_list)
    )

In [11]:
# Process POLARIS Raster Image Part 1 of 2

# Create function with description to process raster images
def process_image(url, soil_prop, soil_stat, soil_depth, bounds_gdfs, output_dir):
    """
    Load, crop, and scale raster images for multiple sites.

    Parameters
    ----------
    url: str
      URL or path for raster files.
    soil_prop: str
      Soil property (e.g., "sand", "clay", etc.)
    soil_stat: str
      Soil statistic (e.g., "mean", "median", etc.)
    soil_depth: str
      Soil depth (e.g., "30-60cm", "60-100cm", etc.)
    bounds_gdf: gpd.GeoDataFrame
      Area of interest to crop to.

    Returns
    -------
   all_merged_da_list: rxr.DataArray
      List of processed rasters from multiple sites.
   saved_files: rxr.DataArray
      saved processed rasters from multiple sites.
    """

    # List to store merged rasters for all sites and areas
    all_merged_da_list = []  

    # List to store paths of saved raster files
    saved_files = []  

    # Iterate through the list of bounding GeoDataFrames (areas of interest)
    for bounds_gdf in bounds_gdfs:

      # Get the study bounds
      bounds_min_lon, bounds_min_lat, bounds_max_lon, bounds_max_lat = (
      bounds_gdf
      .to_crs(4326)
      .total_bounds 
      )

      # List to store cropped DataArrays for the current site
      da_list = []
      
      # Loop through bounding box coordinates
      for min_lon in range(floor(bounds_min_lon), ceil(bounds_max_lon)):
        for min_lat in range(floor(bounds_min_lat), ceil(bounds_max_lat)):

          # Format the URL with the current coordinates and other parameters
          formated_url = (
            url.format( 
                soil_prop = soil_prop, 
                soil_stat = soil_stat, 
                soil_depth = soil_depth,
                min_lat=min_lat , max_lat=min_lat+1,
                min_lon=min_lon, max_lon=min_lon+1 )
          )

          # Connect to the raster image
          da = rxr.open_rasterio(
          formated_url, 
          mask_and_scale=True
          ).squeeze()
          
          # Crop the raster image to the bounds of the study area
          cropped_da = (
          da.rio.clip_box(bounds_min_lon, bounds_min_lat, bounds_max_lon, bounds_max_lat)
          )

          # Append the cropped DataArray to the list
          da_list.append(cropped_da)

      # Merge the cropped DataArrays for this site
      merged_da_list = (
      merge_arrays(da_list)
      )
      
      # Save the merged raster to the output directory
      output_file = os.path.join(output_dir, f"merged_raster_{soil_prop}_{soil_stat}_{soil_depth}_area{bounds_gdf}.tif")
      merged_da_list.rio.to_raster(output_file)

      # Print confirmation message
      print(f"Merged raster saved to: {output_file}")

      # Append the file path to the list of saved files
      saved_files.append(output_file)

      # Append the merged DataArray for this site and area to the list
      all_merged_da_list.append(merged_da_list)

      return all_merged_da_list, saved_files


In [ ]:
# Process POLARIS raster image part 2 of 2
# Test the function by defining variables and plotting

# Set the site parameters
# soil variables
soil_prop = 'ph'
soil_stat = 'mean'
soil_depth = '60-100'
# set up url template
soil_url_template = (
            "http://hydrology.cee.duke.edu"
            "/POLARIS/PROPERTIES/v1.0"
            "/{soil_prop}"
            "/{soil_stat}"
            "/{soil_depth}"
            "/lat{min_lat}{max_lat}_lon{min_lon}{max_lon}.tif"
            )


# output_directory - create data dir for polaris data 
polaris_dir= os.path.join(habitat_suitability_data_dir, 'polaris')
os.makedirs(polaris_dir, exist_ok=True)

# bounds_gdfs
#comanche_grassland_gdf = gpd.read_file(os.path.join(data_dir, 'path_to_shapefile_area1.shp'),
#pawnee_grassland_gdf = gpd.read_file(os.path.join(data_dir, 'path_to_shapefile_area2.shp')
chosen_grasslands_bounds_gdfs = [comanche_grassland_gdf, pawnee_grassland_gdf]

# Test function
#comanche_polaris_processed_image = 
process_image(
   soil_url_template, 
   soil_prop, soil_stat, soil_depth, 
   chosen_grasslands_bounds_gdfs,
   polaris_dir
   )
#)

In [ ]:
# call the variable to check location
polaris_dir

In [ ]:
# Plot
process_image(
    soil_url_template, 
    soil_prop, soil_stat, soil_depth, 
    chosen_grasslands_bounds_gdfs[0],
    
    ).plot(
    cbar_kwargs={"label": "pH"},
    robust=True,
)
plt.gca().set(
    title='Comanche National Grassland - pH ',
    xlabel='Longitude', 
    ylabel='Latitude',
    xticks=[],
    yticks=[]
)
plt.show()